# Hive
![Hive](https://hive.apache.org/images/hive_logo_medium.jpg)

- https://hive.apache.org/

## Setup

- version 3.1.2

In [1]:
%%bash

# Download package
cd /opt/pkgs
wget -q -c https://downloads.apache.org/hive/hive-3.1.2/apache-hive-3.1.2-bin.tar.gz

# unpack file and create link
tar -zxf apache-hive-3.1.2-bin.tar.gz -C /opt
ln -s /opt/apache-hive-3.1.2-bin /opt/hive

# update envvars.sh
cat >> /opt/envvars.sh << EOF
# Hive
export HIVE_HOME=/opt/hive
export PATH=\${PATH}:\${HIVE_HOME}/bin

EOF

# Fix guava and slf4j versions
rm /opt/hive/lib/guava-19.0.jar
cp /opt/hadoop/share/hadoop/hdfs/lib/guava-27.0-jre.jar /opt/hive/lib
rm /opt/hive/lib/log4j-slf4j-impl-2.10.0.jar

cat /opt/envvars.sh

export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh

export HADOOP_HOME=/opt/hadoop
export HADOOP_COMMON_HOME=${HADOOP_HOME}
export HADOOP_CONF_DIR=${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=${HADOOP_HOME}
export HADOOP_MAPRED_HOME=${HADOOP_HOME}
export HADOOP_YARN_HOME=${HADOOP_HOME}

export PATH=${PATH}:${HADOOP_HOME}/bin:${HADOOP_HOME}/sbin     

# Flume
export FLUME_HOME=/opt/flume
export PATH=${PATH}:${FLUME_HOME}/bin

# Sqoop
export SQOOP_HOME=/opt/sqoop
export PATH=${PATH}:${SQOOP_HOME}/bin

# Pig
export PIG_HOME=/opt/pig
export PATH=${PATH}:${PIG_HOME}/bin

# Hive
export HIVE_HOME=/opt/hive
export PATH=${PATH}:${HIVE_HOME}/bin



In [2]:
# Load environment variables
%load_ext dotenv
%dotenv -o /opt/envvars.sh
%env

{'HOSTNAME': 'hadoop',
 'OLDPWD': '/',
 'PWD': '/opt',
 'HOME': '/home/hadoop',
 'SHELL': '/bin/bash',
 'SHLVL': '1',
 'PATH': '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/hadoop/bin:/opt/hadoop/sbin:/opt/flume/bin:/opt/sqoop/bin:/opt/pig/bin:/opt/hive/bin',
 '_': '/usr/bin/nohup',
 'LANGUAGE': 'en.UTF-8',
 'LANG': 'en.UTF-8',
 'JPY_PARENT_PID': '1566',
 'TERM': 'xterm-color',
 'CLICOLOR': '1',
 'PAGER': 'cat',
 'GIT_PAGER': 'cat',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'JAVA_HOME': '/usr/lib/jvm/java-1.8.0-openjdk-amd64',
 'PDSH_RCMD_TYPE': 'ssh',
 'HADOOP_HOME': '/opt/hadoop',
 'HADOOP_COMMON_HOME': '/opt/hadoop',
 'HADOOP_CONF_DIR': '/opt/hadoop/etc/hadoop',
 'HADOOP_HDFS_HOME': '/opt/hadoop',
 'HADOOP_MAPRED_HOME': '/opt/hadoop',
 'HADOOP_YARN_HOME': '/opt/hadoop',
 'FLUME_HOME': '/opt/flume',
 'SQOOP_HOME': '/opt/sqoop',
 'PIG_HOME': '/opt/pig',
 'HIVE_HOME': '/opt/hive'}

## Hadoop configuration (for beeline)

- core-site.xml

```xml
<configuration>
...
<property>
  <name>hadoop.proxyuser.hadoop.groups</name>
  <value>*</value>
</property>
<property>
  <name>hadoop.proxyuser.hadoop.hosts</name>
  <value>*</value>
</property>
</configuration>
```

## Hive Metastore

- using local Derby database

### Create directory in HDFS

In [3]:
%%bash

hdfs dfs -mkdir -p /user/hive/warehouse
hdfs dfs -chmod g+w /user/hive/warehouse

### Initialize database

In [4]:
%%bash

mkdir $HIVE_HOME/hiveserver2
cd $HIVE_HOME/hiveserver2
$HIVE_HOME/bin/schematool -dbType derby -initSchema 2> /dev/null

Metastore connection URL:	 jdbc:derby:;databaseName=metastore_db;create=true
Metastore Connection Driver :	 org.apache.derby.jdbc.EmbeddedDriver
Metastore connection User:	 APP
Starting metastore schema initialization to 3.1.0
Initialization script hive-schema-3.1.0.derby.sql
Initialization script completed
schemaTool completed


### Start hiveserver2

In [5]:
%%bash

cd /opt/hive/hiveserver2
nohup /opt/hive/bin/hive --service hiveserver2 \
--hiveconf hive.security.authorization.createtable.owner.grants=ALL \
--hiveconf hive.root.logger=INFO,console > hiveserver2.out 2>&1 &
echo $! > hiveserver2.pid

## Example

- SF Bay Area Bike Share (https://www.kaggle.com/benhamner/sf-bay-area-bike-share)
- stations.csv and trips.csv

In [6]:
%%bash

mkdir /opt/datasets/bikeshare
tar -zxf hivedataset.tgz -C /opt/datasets/bikeshare

cd /opt/datasets/bikeshare
ls

hdfs dfs -mkdir -p bikeshare/stations
hdfs dfs -put stations.csv bikeshare/stations
hdfs dfs -mkdir -p bikeshare/trips
hdfs dfs -put trips.csv bikeshare/trips

stations.csv
trips.csv


2021-01-29 16:53:29,360 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-29 16:53:41,312 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-29 16:53:43,882 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


## Connect with beeline

1. Run beeline

```bash
source /opt/envvars.sh
beeline -n hadoop -u jdbc:hive2://localhost:10000
```

2. Configure jobs executor

```sql
SET hive.execution.engine=mr;
SET mapreduce.framework.name=yarn;
```

3. Create bikeshare database

```sql
CREATE DATABASE bikeshare;
SHOW DATABASES;
USE bikeshare;
```

4. Create stations table

```sql
CREATE EXTERNAL TABLE stations (
station_id INT,
name STRING,
lat DOUBLE,
long DOUBLE,
dockcount INT,
landmark STRING,
installation STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs:///user/hadoop/bikeshare/stations';
```

5. Create trips table

```sql
CREATE EXTERNAL TABLE trips (
trip_id INT,
duration INT,
start_date STRING,
start_station STRING,
start_terminal INT,
end_date STRING,
end_station STRING,
end_terminal INT,
bike_num INT,
subscription_type STRING,
zip_code STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs:///user/hadoop/bikeshare/trips';
```

6. Show tables

```sql
SHOW TABLES;
DESCRIBE stations;
DESCRIBE trips;
DESCRIBE FORMATTED stations;
DESCRIBE FORMATTED trips;
```

7. Run query - number of trips per terminal

```sql
SELECT start_terminal, start_station, COUNT(1) AS count
FROM trips
GROUP BY start_terminal, start_station
ORDER BY count
DESC LIMIT 10;
```

8. Run query - join between stations and trips

```sql
SELECT t.trip_id, t.duration, t.start_date, s.name, s.lat, s.long, s.landmark
FROM stations s
JOIN trips t ON s.station_id = t.start_terminal
LIMIT 10;
```

9. Exit beeline

```sql
!quit
```

In [7]:
%%bash

cd /opt/hive/hiveserver2

# kill hiveserver2
kill $(cat hiveserver2.pid)
rm hiveserver2.pid